In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from pricer.evaluator import evaluate
from pricer.items import Item
import joblib

In [2]:
username = "kshitijchaudhary"
dataset = f"{username}/items_lite"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


In [3]:
def random_pricer(item):
    return random.randrange(1, 1000)

In [4]:
random.seed(42)
evaluate(random_pricer, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$436 $1 $29 $690 $252 $21 $85 $72 $719 $225 $20 $380 $894 $505 $11 $572 $354 $17 $179 $23 $90 $115 $433 $442 $304 $122 $291 $714 $567 $639 $539 $370 $66 $380 $489 $534 $769 $835 $207 $740 $626 $84 $680 $178 $129 $260 $142 $189 $836 $580 $310 $25 $380 $270 $47 $234 $861 $313 $417 $259 $591 $33 $657 $361 $79 $38 $757 $500 $263 $5 $534 $284 $570 $625 $584 $871 $759 $361 $575 $178 $602 $60 $17 $579 $207 $732 $115 $224 $756 $193 $866 $9 $370 $250 $456 $423 $821 $217 $103 $195 $264 $98 $650 $135 $470 $842 $675 $264 $43 $325 $591 $138 $516 $619 $56 $2 $449 $369 $221 $845 $640 $616 $501 $59 $502 $273 $844 $688 $616 $81 $164 $705 $52 $795 $259 $396 $70 $2 $89 $798 $902 $331 $818 $716 $129 $186 $627 $2 $141 $873 $918 $553 $423 $7 $253 $136 $204 $707 $255 $502 $19 $739 $557 $426 $80 $575 $39 $321 $185 $132 $497 $484 $326 $751 $53 $733 $85 $64 $549 $71 $158 $672 $133 $362 $16 $373 $258 $544 $420 $515 $223 $944 $532 $743 $866 $68 $527 $459 $67 $673 

In [7]:
training_prices = [item.price for item in train]
avg_price = sum(training_prices) / len(training_prices)
print(avg_price)

140.347293000021


In [11]:
def average_training_price(item):
    return avg_price

In [13]:
evaluate(average_training_price, train)

  0%|          | 0/200 [00:00<?, ?it/s]

$76 $61 $100 $309 $60 $50 $35 $66 $100 $115 $144 $61 $590 $100 $55 $60 $110 $65 $52 $80 $113 $132 $260 $122 $59 $97 $56 $93 $180 $125 $56 $90 $315 $604 $101 $126 $20 $79 $60 $111 $105 $580 $105 $19 $106 $169 $82 $50 $89 $70 $119 $100 $41 $58 $96 $120 $111 $261 $30 $113 $43 $72 $370 $40 $75 $45 $15 $89 $97 $4 $120 $107 $20 $190 $678 $18 $53 $117 $104 $30 $104 $124 $116 $112 $117 $107 $120 $38 $80 $57 $92 $2 $60 $60 $51 $11 $96 $62 $94 $71 $41 $4 $30 $122 $102 $31 $123 $74 $117 $72 $111 $64 $55 $10 $120 $95 $72 $260 $130 $111 $210 $127 $238 $111 $98 $6 $629 $80 $66 $460 $724 $159 $80 $90 $93 $116 $155 $41 $110 $291 $100 $98 $129 $100 $78 $85 $55 $103 $30 $29 $122 $11 $20 $128 $54 $115 $129 $40 $40 $129 $120 $19 $121 $111 $10 $122 $274 $110 $3 $108 $160 $10 $140 $103 $91 $59 $94 $102 $561 $96 $105 $47 $78 $131 $34 $756 $113 $124 $102 $71 $91 $10 $37 $119 $350 $64 $253 $50 $5 $60 

In [16]:
def get_features(item):
    return {
        "weight": item.weight,
        "weight_unknown": 1 if item.weight ==0 else 0,
        "text_length": len(item.summary)
    }

In [25]:
def list_to_dataframe(items):
    features = [get_features(item) for item in train]
    df = pd.DataFrame(features)
    df['price'] = [item.price for item in train]
    return df

train_df = list_to_dataframe(train)
test_df = list_to_dataframe(test)

In [26]:
train_df

,weight,weight_unknown,text_length,price
0,1.500000,0,308,64.30
1,0.485017,0,312,79.00
2,0.520000,0,359,240.00
3,2.270000,0,480,449.00
4,0.525000,0,428,79.99
...,...,...,...,...
19995,15.000000,0,371,82.72
19996,0.287500,0,315,149.00
19997,0.000625,0,378,50.56
19998,0.136250,0,366,35.00


In [27]:
random.seed(42)

feature_columns = ['weight', 'weight_unknown', 'text_length']

X_train = train_df[feature_columns]
y_train = train_df['price']
X_test = test_df[feature_columns]
y_test = test_df['price']

model = LinearRegression()
model.fit(X_train, y_train)

for feature, coeff in zip(feature_columns, model.coef_):
    print(f"{feature}: {coeff}")
print(f"Intercept: {model.intercept_}")

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Square Error is: {mse}")
print(f"R-Squared score is: {r2}")



weight: 3.5676099219759356
weight_unknown: 20.49967303604297
text_length: 0.20714449042916366
Intercept: 39.75504757399429
Mean Square Error is: 21774.673894264604
R-Squared score is: 0.1582639698804249


In [63]:
def linear_regression(item):
    features = [get_features(item)]
    df = pd.DataFrame(features)
    print(df)
    return model.predict(df)[0]

linear_regression(test[0])

   weight  weight_unknown  text_length
0     2.0               0          468


np.float64(143.83388893879476)

In [36]:
evaluate(linear_regression, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$75 $51 $74 $49 $60 $37 $13 $48 $93 $195 $577 $231 $103 $80 $25 $78 $50 $56 $43 $10 $17 $13 $73 $22 $190 $328 $349 $103 $46 $19 $107 $94 $42 $9 $89 $664 $67 $44 $83 $77 $70 $46 $78 $73 $97 $92 $104 $92 $31 $55 $85 $2 $344 $7 $69 $24 $104 $88 $39 $122 $108 $36 $23 $6 $238 $29 $99 $267 $4 $94 $83 $117 $166 $88 $71 $65 $6 $93 $104 $87 $75 $88 $86 $11 $81 $72 $77 $157 $74 $54 $93 $52 $92 $81 $89 $108 $94 $12 $123 $415 $39 $15 $71 $30 $115 $5 $85 $301 $99 $169 $47 $148 $90 $63 $74 $55 $111 $85 $87 $22 $85 $39 $71 $57 $133 $62 $96 $97 $58 $2 $47 $3 $33 $75 $52 $89 $79 $39 $14 $2 $55 $130 $42 $88 $12 $80 $52 $360 $8 $78 $83 $10 $73 $6 $84 $126 $112 $33 $33 $81 $163 $99 $75 $127 $720 $106 $300 $75 $81 $122 $80 $96 $250 $74 $150 $68 $90 $6 $60 $31 $287 $105 $51 $85 $62 $102 $15 $60 $62 $98 $79 $55 $102 $31 $14 $100 $54 $116 $81 $114 